In [19]:
from collections import defaultdict

In [1]:
exit_price = 30
fin_independence = 3
redistribution_tax = .5

#Shareholders not subject to progressive equity, such as investors{name: % equity, }
std_equity = {'angel1': [.1, 0] , 'venture1': [.1, 0] }
std_equity_definitions = ['Equity Percent', 'Standard Payout']

#Shareholders that are subject to progressive equity{name: % equity, eligible for redistribution(boolean)}
prog_equity = {'founder1': [0.3, 0, False, True, 0, 0] , 
               'founder2' : [0.3, 0, False, True, 0, 0] , 
               'e1': [.075, 0, False, True, 0, 0], 
               'e2' : [.025, 0, False, True, 0, 0], 
               'e3' : [.05, 0, False, False, 0, 0], 
               'e4' : [.05, 0, False, False, 0, 0]}
prog_equity_definitions = ['Equity Percent',  'Standard Payout Amount', 
                           'Triggered for Equity Tax', 'Eligible for Redistribution',
                           'Progressive Payout Amount', 'Percent of Total Payout']

In [16]:
def standard_payout(exit_price, std_equity, prog_equity):
    '''
    Calculate payout that each shareholder would receive under a standard equity distribution.
    
    Parameters
    ----------
    
    Results
    -------
    
    
    ''' 
    for name, data in std_equity.items():
        equity_percent = data[0]
        std_equity[name][1] = (equity_percent * exit_price)
    
    
    for name, data in prog_equity.items():
        equity_percent = data[0]
        prog_equity[name][1] = (equity_percent * exit_price)
    
    return std_equity, prog_equity

In [ ]:
std_equity, prog_equity = standard_payout(exit_price, std_equity, prog_equity)


In [ ]:
std_equity, prog_equity

In [6]:
def triggered_list(prog_equity, fin_independence, redistribution_tax):
    '''
    Check if progressive equity holder will pass financial independence number and
    be triggered for equity tax.
    
    '''
    #Keeps track of who has been triggered in this round of calculations
    triggered_current = defaultdict(list)
    
    for name, data in prog_equity.items():
        standard_payout = data[1]
        triggered = data[2]
        if standard_payout > fin_independence and triggered == False: 
            redistribution_payout = (standard_payout - fin_independence) * redistribution_tax
            triggered_current[name] = redistribution_payout
            prog_equity[name][2] = True
            prog_equity[name][4] = (standard_payout - redistribution_payout)
    
    return triggered_current, prog_equity

In [ ]:
triggered_current, prog_equity = triggered_list(prog_equity, fin_independence, redistribution_tax)
triggered_current, prog_equity

In [7]:
def contribute_pool(triggered_current):
    '''
    Equity holders that have been 'triggered' make a contribution to the redistribution pool.
    
    '''
    
    redistribution_pool = 0
    
    for values in triggered_current.values():
        redistribution_pool += values
    
    return redistribution_pool
        
        
    
    
    
    

In [ ]:
redistribution_pool = contribute_pool(triggered_current)
redistribution_pool

In [8]:
def redistribution_denominator(prog_equity):
    '''
    Calculate denominator that will be used to determine the share of the redistribution pool
    received by each employee. 
    
    
    '''
    total_equity = 0
    for data in prog_equity.values():
        equity_percent = data[0]
        triggered = data[2]
        if triggered == False:
            total_equity += equity_percent
    return total_equity
            

In [ ]:
redistribution_denominator = redistribution_denominator(prog_equity)
redistribution_denominator

In [ ]:
redistribution_denominator = 0
redistribution_denominator

In [9]:
def distribute_pool(prog_equity, redistribution_pool, redistribution_denominator):
    '''
    Distribute funds in pool to eligible employees based on their proportion of the equity
    eligible for redistribution. The equity eligible for redistribution is the "redistribution
    denominator".
    
    '''
    for name, data in prog_equity.items():
        equity_percent = data[0]
        std_payout_amount = data[1]
        triggered = data[2]
        eligible = data[3]
        if triggered == False and eligible == True:
            redistribution_percent = equity_percent / redistribution_denominator
            redistribution_received = redistribution_percent * redistribution_pool
            prog_payout = redistribution_received + std_payout_amount
            prog_equity[name][4] = std_payout_amount + prog_payout
        elif triggered == False and eligible == False:
            prog_equity[name][4] = std_payout_amount
        
    return prog_equity

            
    
    

In [ ]:
prog_equity = distribute_pool(prog_equity, redistribution_pool, redistribution_denominator)
prog_equity

In [10]:
def test(std_equity, prog_equity, exit_price):
    '''
    Test to check that total payout equals the exit price. 
    
    '''
    test_exit_price = 0
    for values in std_equity.values():
        std_payout = values[1]
        test_exit_price += std_payout
    for values in prog_equity.values():
        prog_payout = values[4]
        test_exit_price += prog_payout
    
    if test_exit_price == exit_price:
        return True
    else:
        return False
    
    

In [ ]:
test_result = test(std_equity, prog_equity, exit_price)
test_result

In [11]:
def percent_final_payout(prog_equity, exit_price):
    '''
    Calculate the percent of the total exit price received by each progressive equity holder.
    
    '''
    for name, data in prog_equity.items():
        prog_equity[name][5] = data[4] / exit_price
    return prog_equity
    

In [ ]:
prog_equity = percent_final_payout(prog_equity, exit_price)
prog_equity

In [12]:
std_equity

{'angel1': [0.1, 0], 'venture1': [0.1, 0]}

In [13]:
prog_equity

{'e1': [0.075, 0, False, True, 0, 0],
 'e2': [0.025, 0, False, True, 0, 0],
 'e3': [0.05, 0, False, False, 0, 0],
 'e4': [0.05, 0, False, False, 0, 0],
 'founder1': [0.3, 0, False, True, 0, 0],
 'founder2': [0.3, 0, False, True, 0, 0]}

In [20]:
def calculate_payouts(exit_price, fin_independence, redistribution_tax, std_equity, prog_equity):
    '''
    Calculate payouts for each equity holder in a progressive equity scheme. 
    
    '''
        
    #Calculate the standard payout
    std_equity_1, prog_equity_1 = standard_payout(exit_price, std_equity, prog_equity)
    print(type(prog_equity_1))
    
    #Determine who would be triggered for the equity tax
    triggered_current, prog_equity_1 = triggered_list(prog_equity_1, fin_independence, redistribution_tax)
    print(type(prog_equity_1))
    
    #Contribute funds to the redistrubtion pool
    redistribution_pool = contribute_pool(triggered_current)
    print(type(prog_equity_1))
    
#     #Calculate the denominator used to determine the distribution of the pool
#     denominator = redistribution_denominator(prog_equity=prog_equity_1)
    
#     #Distribute the pool to eligible equity holders
#     prog_equity_1 = distribute_pool(prog_equity_1, redistribution_pool, denominator)
    
#     #Calculate the percent of the total exit price received by each progressive equity holder
#     prog_equity_1 = percent_final_payout(prog_equity_1, exit_price)
    
#     #Check that total payouts equal the exit price
#     test_result = test(std_equity_1, prog_equity_1, exit_price)
    
#     if test_result == False:
#         print("The total payouts do not equal the exit price. There is an error.")
        
    return std_equity_1, prog_equity_1
        
    
    

In [21]:
calculate_payouts(exit_price, fin_independence, redistribution_tax, std_equity, prog_equity)

<class 'dict'>
<class 'dict'>
<class 'dict'>


({'angel1': [0.1, 3.0], 'venture1': [0.1, 3.0]},
 {'e1': [0.075, 2.25, False, True, 0, 0],
  'e2': [0.025, 0.75, False, True, 0, 0],
  'e3': [0.05, 1.5, False, False, 0, 0],
  'e4': [0.05, 1.5, False, False, 0, 0],
  'founder1': [0.3, 9.0, True, True, 6.0, 0],
  'founder2': [0.3, 9.0, True, True, 6.0, 0]})

In [22]:
std_equity

{'angel1': [0.1, 3.0], 'venture1': [0.1, 3.0]}

In [23]:
prog_equity

{'e1': [0.075, 2.25, False, True, 0, 0],
 'e2': [0.025, 0.75, False, True, 0, 0],
 'e3': [0.05, 1.5, False, False, 0, 0],
 'e4': [0.05, 1.5, False, False, 0, 0],
 'founder1': [0.3, 9.0, True, True, 6.0, 0],
 'founder2': [0.3, 9.0, True, True, 6.0, 0]}